<a href="https://colab.research.google.com/github/JuanFelipeSanchez/Data-Scientist/blob/main/RetoEmpleados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.	Importa las librerías requeridas.
2.	Lee el archivo CSV llamado empleadosRETO.csv y coloca los datos en un frame de Pandas llamado EmpleadosAttrition.

In [99]:
#Carga inicial de conexiones, funciones y librerías.
from google.colab import drive
import pandas as pd
from sklearn.decomposition import PCA
drive.mount('/content/gdrive')

#Lectura del archivo csv.
EmpleadosAttrition = pd.read_csv('/content/gdrive/MyDrive/Data Scientist/15. Ingeniería de características/empleadosRETO.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


3.	Elimina las columnas que, con alta probabilidad (estimada por ti), no tienen relación alguna con la salida. Hay algunas columnas que contienen información que no ayuda a definir el desgaste de un empleado, tal es caso de las siguientes:

a.	EmployeeCount: número de empleados, todos tienen un 1

b.	EmployeeNumber: ID del empleado, el cual es único para cada empleado

c.	Over18: mayores de edad, todos dicen “Y”

d.	StandardHours: horas de trabajo, todos tienen “80”


In [100]:
#Se eliminan las columnas no relacionadas con la salida. EmployeeCount, EmployeeNumber, Over18, StandardHours.
EmpleadosAttrition.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'],axis=1,inplace=True)

4.	Analizando la información proporcionada, detectaste que no se cuenta con los años que el empelado lleva en la compañía y parece ser un buen dato. Dicha cantidad se puede calcular con la fecha de contratación ‘HiringDate’:

a.	Crea una columna llamada Year y obtén el año de contratación del empleado a partir de su fecha ‘HiringDate’. No se te olvide que debe ser un entero.

b.	Crea una columna llamada YearsAtCompany que contenga los años que el empleado lleva en la compañía hasta el año 2018. Para su cálculo, usa la variable Year que acabas de crear.


In [101]:
#Se crea la columna Year. La columna HiringDate se convierte en objeto datetime manejando posibles errores en el formato.
EmpleadosAttrition['HiringDate'] = pd.to_datetime(EmpleadosAttrition['HiringDate'], errors='coerce')
EmpleadosAttrition['Year'] = EmpleadosAttrition['HiringDate'].dt.year


#Se crea la columna YearsAtCompany.
EmpleadosAttrition['YearsAtCompany']=2018-EmpleadosAttrition['Year']

5.	La DistanceFromHome está dada en kilómetros, pero tiene las letras “km” al final y así no puede ser entera:

a.	Renombra la variable DistanceFromHome a DistanceFromHome_km.

b.	Crea una nueva variable DistanceFromHome que sea entera, es decir, solo con números.


In [102]:
#Se renombra la variable DistanceFromHome a DistanceFromHome_km.
EmpleadosAttrition.rename(columns={'DistanceFromHome':'DistanceFromHome_km'},inplace=True)

#Se crea la variable DistanceFromHome.
EmpleadosAttrition['DistanceFromHome']=EmpleadosAttrition['DistanceFromHome_km']
EmpleadosAttrition['DistanceFromHome']=EmpleadosAttrition['DistanceFromHome'].str[:2].astype(int)

6.	Borra las columnas Year, HiringDate y DistanceFromHome_km debido a que ya no son útiles.

In [103]:
#Se eliminan las columnas no útiles. Year, HiringDate y DistanceFromHome_km.
EmpleadosAttrition.drop(['Year', 'HiringDate', 'DistanceFromHome_km'],axis=1,inplace=True)

7.	Aprovechando los ajustes que se están haciendo, la empresa desea saber si todos los departamentos tienen un ingreso promedio similar. Genera una nuevo frame llamado SueldoPromedioDepto que contenga el MonthlyIncome promedio por departamento de los empleados y colócalo en una variable llamada SueldoPromedio. Esta tabla solo es informativa, no la vas a utilizar en el set de datos que estás construyendo.
8.	La variable MonthlyIncome tiene un valor numérico muy grande comparada con las otras variables. Escala dicha variable para que tenga un valor entre 0 y 1.

In [104]:
#Creación de la nueva tabla por agrupación.
SueldoPromedioDepto = EmpleadosAttrition.groupby('Department', as_index=False)['MonthlyIncome'].mean()
SueldoPromedioDepto['MonthlyIncome'] = SueldoPromedioDepto['MonthlyIncome'].round(1)
SueldoPromedioDepto.rename(columns={'MonthlyIncome': 'SueldoPromedio'}, inplace=True)
print(SueldoPromedioDepto)

#Aplicación de escala de 0 a 1 en la variable MonthlyIncome.
EmpleadosAttrition['MonthlyIncome'] = (EmpleadosAttrition['MonthlyIncome'] - EmpleadosAttrition['MonthlyIncome'].min()) / (EmpleadosAttrition['MonthlyIncome'].max() - EmpleadosAttrition['MonthlyIncome'].min())

               Department  SueldoPromedio
0         Human Resources          6239.9
1  Research & Development          6804.1
2                   Sales          7188.2


9.	Todo parece indicar que las variables categóricas que quedan sí son importantes para obtener la variable de salida. Convierte todas las variables categóricas que quedan a numéricas:

BusinessTravel, Department,	EducationField, Gender, JobRole, MaritalStatus, Attrition

In [105]:
#Validación de las variables categóricas
EmpleadosAttrition['BusinessTravel']=EmpleadosAttrition['BusinessTravel'].astype('category')
EmpleadosAttrition['Department']=EmpleadosAttrition['Department'].astype('category')
EmpleadosAttrition['EducationField']=EmpleadosAttrition['EducationField'].astype('category')
EmpleadosAttrition['Gender']=EmpleadosAttrition['Gender'].astype('category')
EmpleadosAttrition['JobRole']=EmpleadosAttrition['JobRole'].astype('category')
EmpleadosAttrition['MaritalStatus']=EmpleadosAttrition['MaritalStatus'].astype('category')
EmpleadosAttrition['Attrition']=EmpleadosAttrition['Attrition'].astype('category')

#Transformación de variables
EmpleadosAttrition['BusinessTravelNum']=EmpleadosAttrition['BusinessTravel'].cat.codes
EmpleadosAttrition['DepartmentNum']=EmpleadosAttrition['Department'].cat.codes
EmpleadosAttrition['EducationFieldNum']=EmpleadosAttrition['EducationField'].cat.codes
EmpleadosAttrition['GenderNum']=EmpleadosAttrition['Gender'].cat.codes
EmpleadosAttrition['JobRoleNum']=EmpleadosAttrition['JobRole'].cat.codes
EmpleadosAttrition['MaritalStatusNum']=EmpleadosAttrition['MaritalStatus'].cat.codes
EmpleadosAttrition['AttritionNum']=EmpleadosAttrition['Attrition'].cat.codes

10.	Ahora debes hacer la evaluación de las variables para quedarte con las mejores. Calcula la correlación lineal de cada una de las variables con respecto al Attrition.

In [106]:
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['Age']),'Age')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['Education']),'Education')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['EnvironmentSatisfaction']),'EnvironmentSatisfaction')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['JobInvolvement']),'JobInvolvement')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['JobLevel']),'JobLevel')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['JobSatisfaction']),'JobSatisfaction')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['MonthlyIncome']),'MonthlyIncome')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['NumCompaniesWorked']),'NumCompaniesWorked')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['PercentSalaryHike']),'PercentSalaryHike')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['PerformanceRating']),'PerformanceRating')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['RelationshipSatisfaction']),'RelationshipSatisfaction')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['TotalWorkingYears']),'TotalWorkingYears')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['TrainingTimesLastYear']),'TrainingTimesLastYear')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['WorkLifeBalance']),'WorkLifeBalance')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['YearsInCurrentRole']),'YearsInCurrentRole')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['YearsSinceLastPromotion']),'YearsSinceLastPromotion')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['BusinessTravelNum']),'BusinessTravelNum')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['DepartmentNum']),'DepartmentNum')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['EducationFieldNum']),'EducationFieldNum')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['GenderNum']),'GenderNum')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['JobRoleNum']),'JobRoleNum')
print(EmpleadosAttrition['AttritionNum'].corr(EmpleadosAttrition['MaritalStatusNum']),'MaritalStatusNum')

-0.21212111259206842 Age
-0.05553079310984968 Education
-0.12432694416927641 EnvironmentSatisfaction
-0.16678478967808572 JobInvolvement
-0.21426604317577613 JobLevel
-0.16495679045325196 JobSatisfaction
-0.19493601628571225 MonthlyIncome
-0.009081775325922558 NumCompaniesWorked
-0.06087967966387507 PercentSalaryHike
-0.00647103495364531 PerformanceRating
-0.03094453065814358 RelationshipSatisfaction
-0.21332903741456125 TotalWorkingYears
-0.07088383377930298 TrainingTimesLastYear
-0.021722932349419282 WorkLifeBalance
-0.2039180268902555 YearsInCurrentRole
-0.06900034357702403 YearsSinceLastPromotion
0.08289944709936396 BusinessTravelNum
0.05423584846525848 DepartmentNum
0.051184319893178454 EducationFieldNum
-0.02883870932201118 GenderNum
0.07868359123827778 JobRoleNum
0.18040360207821074 MaritalStatusNum


11.	Selecciona solo aquellas variables que tengan una correlación mayor o igual a 0.1, dejándolas en otro frame llamado EmpleadosAttritionFinal. No olvides mantener la variable de salida Attrition; esto es equivalente a borrar las que no cumplen con el límite.

In [107]:
EmpleadosAttritionFinal = EmpleadosAttrition[['AttritionNum','Age','EnvironmentSatisfaction','JobInvolvement','JobLevel','JobSatisfaction','MonthlyIncome','TotalWorkingYears','YearsInCurrentRole','MaritalStatusNum']]

12.	Crea una nueva variable llamada EmpleadosAttritionPCA formada por los componentes principales del frame EmpleadosAttritionFinal. Recuerda que el resultado del proceso PCA es un numpy array, por lo que, para hacer referencia a una columna, por ejemplo, la 0, puedes usar la instrucción EmpleadosAttritionPCA[:,0]).

In [124]:
#Se crea la nueva variable PCA
EmpleadosAttritionPCA = EmpleadosAttritionFinal
pca = PCA(n_components=0.90)
pca.fit(EmpleadosAttritionPCA)

print('Varianza explicada por los componentes')
print(pca.explained_variance_ratio_)

EmpleadosAttritionPCA = pca.transform(EmpleadosAttritionPCA)

Varianza explicada por los componentes
[0.72843557 0.18126712]


13.	Agrega el mínimo número de Componentes Principales en columnas del frame EmpleadosAttritionPCA que logren explicar el 80% de la varianza, al frame EmpleadosAttritionFinal. Puedes usar la instrucción assign, columna por columna, llamando a cada una C0, C1, etc., hasta las que vayas a agregar.

In [133]:
#Determinación de cantidad de componentes necesarios para explicar al menos el 80%
varianza_explicada = pca.explained_variance_ratio_
varianza_acumulada = varianza_explicada.cumsum()
componentes_minimos = (varianza_acumulada >= 0.80).argmax() + 1
print('Cantidad de componentes necesarios: ',componentes_minimos)

# Seleccionar de componentes principales
componentes_seleccionados = EmpleadosAttritionPCA[:, :componentes_minimos]

# Crear columnas C0, C1, ..., C(n-1)
component_columns = {f'C{i}': componentes_seleccionados[:, i] for i in range(componentes_minimos)}

# Agregar los componentes al DataFrame original
EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(**component_columns)

Cantidad de componentes necesarios:  2


14.	Guarda el set de datos que has formado y que tienes en EmpleadosAttritionFinal en un archivo CSV llamado EmpleadosAttritionFinal.csv

In [149]:
from google.colab import files

EmpleadosAttritionFinal.to_csv('EmpleadosAttritionFinal.csv', index=False)
files.download('EmpleadosAttritionFinal.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>